# M02E14: Machine Learning Exercises 2 - Sample Solutions

---

## **Task 1** - Data Preparation for Income Prediction

An individual's annual income results from various factors. Intuitively, it is influenced by the individual's education level, age, gender, occupation, and etc. To understand such an influence, Ronny Kohavi and Barry Becker (Data Mining and Visualization, Silicon Graphics) constructed the Adult dataset (`adult.csv`, located at your `Data` directory), extracted from the 1994 US Census bureau database. The dataset consists of 6 continuous attributes and 8 nominal (categorical) ones. 

Conduct an exploratory data analysis on this dataset to answer the questions below.

1. Among whom is the proportion of those who earn a lot (>50K) greater: married or single men (marital.status feature)? Consider as married those who have a marital-status starting with Married (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.
2. Count the average time of work (hours.per.week) for each country (native-country). What will these be for Germany?
3. Is it true that people who receive more than 50k have at least high school education? (education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate feature)

With the insights gained from the conducted analysis, **build a classifier that accurately determines whether a person makes over 50K US dollars/year**. Note that the dataset contains missing values and categorical data that must be handled before you start building your classifier.

**Solution sketch**

* For tasks 1 to 3, simply use methods like `.loc()`, `.value_counts()`, and `.unique()` from pandas dataframe.
* To build a classifier, you should first handle missing values (e.g., impute mean/mode values or exclude records). Then, transform categorical data using one-hot encoding. Finally, apply feature scaling and create train/test sets.  

In [ ]:
import numpy as np
import pandas as pd
import os

data = pd.read_csv(os.path.join('Data', 'adult.csv'))
data.head()

1. Among whom is the proportion of those who earn a lot (>50K) greater: married or single men (marital-status feature)? Consider as married those who have a marital-status starting with Married (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.

In [ ]:
# Single men

single = ['Never-married', 'Separated', 'Divorced', 'Widowed']

data.loc[(data['sex'] == 'Male') &
    (data['marital.status'].isin(single)), 
    'income'].value_counts()

In [ ]:
# Married men
data.loc[(data['sex'] == 'Male') &
     (data['marital.status'].str.startswith('Married')),
     'income'].value_counts()

2. Count the average time of work (hours.per.week) for those who earn a little and a lot (salary) for each country (native-country). What will these be for Germany?

In [ ]:
pd.crosstab(
    data['native.country'], 
    data['income'], 
    values=data['hours.per.week'], 
    aggfunc=np.mean
)

3. Is it true that people who receive more than 50k have at least high school education? (education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate feature)

In [ ]:
data.loc[data['income'] == '>50K', 'education'].unique() # No

**Build a classifier that accurately determines whether a person makes over 50K US dollars/year.**

Reload the data considering "?" as missing values

In [ ]:
import os
import pandas as pd

data = pd.read_csv(os.path.join('Data', 'adult.csv'), na_values="?")
data.head()

Check for missing values:

In [ ]:
data.isnull().sum()

Plot histograms of columns with missing values:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for c in ['workclass', 'occupation', 'native.country']:
    fig = plt.subplots(figsize=(12, 4))
    sns.histplot(data[c].astype(str))
    plt.xticks(rotation=-35, ha='left')
    plt.show()

Let's separate features from class labels before start preprocessing the data:

In [ ]:
X = data.drop(['income'], axis=1)
y = data.income

Impute missing values using the most frequent value:

In [ ]:
for c in ['workclass', 'occupation', 'native.country']:
    X[c] = X[c].fillna(X[c].mode()[0])

In [ ]:
X.isnull().sum()

Column `education` and `education.num` store the same information.

`education`: an ordered factor with levels Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate.

`education.num`: ordinal categories (numerical) representing education level.

We can keep only one of them. Let's remove column `education`:

In [ ]:
del X['education']

Let's check column data types and encode nominal values into one-hot representations: 

In [ ]:
X.dtypes

In [ ]:
# Apply one-hot encoding

nominal_columns = [c for c in X.columns if X[c].dtype == object]
one_hot_features = pd.get_dummies(X[nominal_columns], drop_first=True, prefix_sep='.')

In [ ]:
# Drop original, nominal features
X = X.drop(nominal_columns, axis=1)

# Add one-hot features
X = X.join(one_hot_features)

In [ ]:
X.head()

Let's now encode class labels as follows:

* `<=50K`: 0
* `>50K`: 1 

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
y = label_enc.fit_transform(y.values)
y

Create sets for training and test. Note that by providing the class label array `y` as an argument to stratify, we ensure that both training and test datasets have the same class proportions as the original dataset.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=0, 
    stratify=y
)

Let's now bring features onto the same scale:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

Now, we can build a classifier. Let's use a decision tree:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(
    criterion='gini', 
    max_depth=10, 
    random_state=1
)

classifier.fit(X_train_std, y_train)

Evaluating training performance:

In [ ]:
import sklearn.metrics as metrics

print(metrics.classification_report(y_train, classifier.predict(X_train_std)))

Evaluating test performance:

In [ ]:
import sklearn.metrics as metrics

print(metrics.classification_report(y_test, classifier.predict(X_test_std)))

Plotting confusion matrix for test data:

In [ ]:
metrics.plot_confusion_matrix(
    classifier, 
    X_test_std,
    y_test, 
    cmap="bone", 
    values_format='d'
);

## **Task 2** - Dimensionality Reduction

Computer vision is an interdisciplinary field that deals with how computers can gain high-level understanding from digital images or videos. A classic computer vision problem is the identification of hand-written digits, that is, identifying characters in an image.

Scikit-Learn includes a dataset of pre-formatted digits, which is built into the library. The digits dataset consists of 8x8 pixel images of digits. `digits.images` stores 8x8 arrays of grayscale values for each image. `digits.target` stores the digit each image represents.

Given the **digits dataset** accessible from `from sklearn.datasets import load_digits`, solve the problems below.

1. Build a classifier that accurately identifies 10 classes where each class refers to a digit. To apply a classifier on this data, you need to flatten the images, turning each 2-D array of grayscale values from shape (8, 8) into shape (64,). Subsequently, the entire dataset will be of shape (n_samples, n_features), where n_samples is the number of images and n_features is the total number of pixels in each image. Data in such a format is already available with `digits.data`.
2. Since the data dimensionality is large (8x8 = 64 features), apply a PCA algorithm to reduce the number of features without losing too much information. 
3. Show how the number of principal components affect classification results.

Use the code block below to load and plot the dataset.

In [ ]:
# Importing packages

import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [ ]:
# Loading the data

digits = load_digits()
digits.keys()

In [ ]:
digits.data # Features

In [ ]:
digits.target # Classes

In [ ]:
# Plotting the images

# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

1. Build a classifier that accurately identifies 10 classes where each class refers to a digit.

https://jakevdp.github.io/PythonDataScienceHandbook/05.08-random-forests.html#Example:-Random-Forest-for-Classifying-Digits

In [ ]:
from sklearn.model_selection import train_test_split

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=0, 
    stratify=y
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(
    criterion='gini', 
    max_depth=10, 
    random_state=1
)

classifier.fit(X_train, y_train)

ypred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

# Define figure size
fig, ax = plt.subplots(figsize=(8, 8))

plot_confusion_matrix(
    classifier, 
    X_test,
    y_test, 
    cmap="bone", 
    values_format='d',
    ax = ax
);

2. Apply a PCA algorithm to reduce the number of features without losing too much information.

The original data is 64-dimensional. However, we can visualize how the data observations are organized into a 2d plot by fitting PCA with only 2 components:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(X_train)

We can now plot the first two principal components of each point to learn about the data:

In [ ]:
# Define figure size
fig = plt.subplots(figsize=(10, 6))

plt.scatter(projected[:, 0], projected[:, 1],
            c=y_train, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('tab10', 10))

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.colorbar();

We can estimate how many components are needed to describe the data by looking at the cumulative explained variance ratio. We can plot a curve that quantifies how much of the total variance is contained within the first **n components**:

In [ ]:
from sklearn.decomposition import PCA

# Train PCA
pca = PCA()
pca.fit(X_train)

# Define figure size
fig = plt.subplots(figsize=(10, 6))

# Plot the cumulative explained variance
plt.plot(np.cumsum(pca.explained_variance_ratio_))

# Show the grid lines
plt.grid()

# Set labels
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

# Define plot x-axis ticks
plt.xticks([x for x in range(0, 65, 5)]);

We see that the first 6 components contain 60% of the variance, while around 45 components are sufficient to describe almost 100% of the variance.

3. Show how the number of principal components affect classification results.

We use a pipeline to train a PCA and subsequently use its components to train a decision tree classifier. We use a StratifiedKFold cross-validation procedure to plot the accuracy of the model trained on a number of components ranging from 2 to 45.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# Define the classifier
classifier = DecisionTreeClassifier(
    criterion='gini', 
    max_depth=10, 
    random_state=1
)

accuracy = []
components = []

for ncomponents in range(2, 46): 
    
    # Define the number of components for PCA
    pca = PCA(ncomponents)
    
    pipe = Pipeline(
        steps=[
            ('pca', pca), 
            ('classifier', classifier)
        ]
    )

    # Define the evaluation procedure
    cv = StratifiedKFold(n_splits=5)
    
    # Compute the cross-validation test accuracy score
    scores = cross_val_score(
        pipe, 
        X, 
        y, 
        scoring='accuracy', 
        cv=cv, 
        n_jobs=-1
    )
    
    # Compute mean accuracy
    accuracy.append(np.mean(scores))
    
    # Add the current value of 'ncomponents' to create a plot.
    components.append(ncomponents)

Plot the obtained accuracy along with the number of components used to trained the decision tree:

In [ ]:
# Define figure size
fig = plt.subplots(figsize=(10, 6))

# Define the plot title
plt.title('Impact of number of components')

# Plot the accuracy score produced by each number of components
plt.plot(components, accuracy)

# Set x and y ticks
plt.xticks([i for i in range(2, 46, 2)])
plt.yticks(np.linspace(0.55, 0.9, 10))

# Show the grid lines
plt.grid();

## **Task 3** - Sentiment Analysis

The use of automatic news text analysis has been growing in computational finance to estimate how financial sentiments relate to future company performance. For this reason, Data4FinanceFuture, a company focused on the use of machine learning to improve financial analysis, hired you as a data scientist. 

Your first assignment in the new job is to conduct a sentiment analysis on top of the **Financial Phrase Bank dataset** (`financialphrasebank.csv`, located at your `Data` directory), collected by researchers associated with the company (check the official [Kaggle webpage](https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news)). The dataset contains sentiments for financial news headlines from the perspective of a retail investor.

The assignment consists of the following tasks:

1. Use a Topic Modeling technique such as LDA to find common topics among the documents/phrases. Find the top 10 words that describe each topic.
2. Build a classifier that accurately identifies sentiments (i.e., neutral, positive, or negative) given a document/phrase.

Use the code block below to load the dataset:

In [ ]:
import numpy as np
import pandas as pd
import os

findata = pd.read_csv(
    os.path.join('Data', 'financialphrasebank.csv'), 
    encoding='latin-1',
    names=['sentiment', 'text'])

findata.head()

**Solution sketch**

1. Clean the data: Remove punctuation and stopwords.
2. Use tokens ans word stemming to prepare the data.
3. Extract features using, for instance, TF-IDF.
4. Train a model (e.g., KNN, SVM, or Decision Trees).
5. Check results using a confusion matrix.


https://www.kaggle.com/shwethaspdevam/comparison-of-models-for-financial-news

Let's start with a quick data exploration phase to gain some insights about the financial news included in the dataset. Check the data summary:

In [ ]:
findata.info()

Check for class imbalance:

In [ ]:
findata['sentiment'].value_counts(ascending=True)

The information above tells us **the dataset is imbalanced**: the `negative` class is two times less frequent the the `positive` class, and three times less as the `neutral` class. That means, to build an accurate classifier, we have to consider the class distribution during the creation of train/test sets.

Compute text length (in number of words):

In [ ]:
findata['text_len'] = findata['text'].apply(lambda t: len(t.split()))

In [ ]:
findata.describe()

Compute total number of words:

In [ ]:
findata['text_len'].sum()

We discovered that the data includes **111948 words** in total, with an **average word count of 23 per article/document**. 

Now, we start preparing the data. Let's start by removing punctuation. Note that we add a new column `prep_text` to store prepared data:

In [ ]:
import re

def remove_punctuation_and_html_lowercase(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [ ]:
findata['prep_text'] = findata['text'].apply(remove_punctuation_and_html_lowercase)

Remove stopwords:

In [ ]:
import nltk

nltk.download('stopwords')
ENGLISH_STOP_WORDS = nltk.corpus.stopwords.words('english')

def remove_stopwords(article):
    # Iterate over every word in an article and filter out stop words
    # We use the .join() method to concatenate strings using an empty space ` ` between them
    return ' '.join([word for word in article.split() if word not in ENGLISH_STOP_WORDS])

In [ ]:
findata['prep_text'] = findata['prep_text'].apply(remove_stopwords)

Recompute statistics:

In [ ]:
findata['prep_text_len'] = findata['prep_text'].apply(lambda t: len(t.split()))
findata.describe()

In [ ]:
findata['prep_text_len'].sum()

We reduced the overall word count from 111948 to **67572 words**, with **average word count around 14 per article/document**. This information is crucial to help us setting hyperparameters later.

Let's check the prepared data:

In [ ]:
findata.head()

1. Topic Modeling via LDA.

In [ ]:
import math
from sklearn.feature_extraction.text import CountVectorizer

# Consider only 10% of the total word count.
num_words = math.floor(.1*findata['prep_text_len'].sum())

count = CountVectorizer(
    stop_words='english',
    max_df=.2, # Max word-doc frequency
    max_features=num_words # Limit number of words
)

X = count.fit_transform(findata['prep_text'].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=5, # Define number of topics
    random_state=123, # For reproducibility
    learning_method='batch' # To speed-up
)

X_topics = lda.fit_transform(X)

In [ ]:
n_top_words = 10
feature_names = count.get_feature_names()

print('---')

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(
        " ".join([
            feature_names[i] for i in topic.argsort()\
            [:-n_top_words - 1:-1]
        ])
    )
    
print('---')

2. Build a classifier that accurately identifies sentiments.

Apply stemming (note that we add another column to stored stemmed text):

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
findata['stemmed_text'] = findata['prep_text'].apply(lambda t: ' '.join(tokenizer_porter(t)))

Let's have a look at the dataframe again:

In [ ]:
findata.head()

Create our features array:

In [ ]:
X = findata['stemmed_text'].values

Let's encode class labels on the `sentiment` column to create a numerical representation:

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
y = label_enc.fit_transform(findata['sentiment'].values)

Now, we create a pipeline that applies SVM over TF-IDF features in a stratified 5-fold cross-validation procedure:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC 

# Consider only 10% of the total word count.
num_words = math.floor(.1*findata['prep_text_len'].sum())

classifier = SVC(kernel = 'rbf', C = 2.0, gamma=0.1)

tfidf = TfidfVectorizer(
    strip_accents=None,
    lowercase=False,
    preprocessor=None,
    max_df=.2, # Max word-doc frequency
    max_features=num_words # Limit number of words)
)

pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', classifier)
])

# Repeats Stratified K-Fold n times 
# with different randomization in each repetition.
# Folds are made by preserving 
# the percentage of samples for each class.
kfoldcv = RepeatedStratifiedKFold(n_splits=5)

scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=kfoldcv, n_jobs=-1)

In [ ]:
scores.mean()

---

# End M02E14:  Machine Learning Exercises 2 - Sample Solutions